# Recommender Project

Dataset [movielens data set](https://grouplens.org/datasets/movielens/) 

In [1]:
import findspark

In [2]:
findspark.init('/home/vlad1974/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [6]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [7]:
data.head(5)

[Row(movieId=2, rating=3.0, userId=0),
 Row(movieId=3, rating=1.0, userId=0),
 Row(movieId=5, rating=2.0, userId=0),
 Row(movieId=9, rating=4.0, userId=0),
 Row(movieId=11, rating=1.0, userId=0)]

In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [10]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [12]:
predictions.show(10)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   3.0|     7|-1.0621548|
|     31|   1.0|    29| 3.7483006|
|     31|   3.0|    14|0.95138377|
|     85|   4.0|     7| 1.2529571|
|     85|   1.0|     2| 1.1856357|
|     78|   1.0|    19| 0.7007147|
|     78|   1.0|    17| 1.1505104|
|     78|   1.0|     8| 1.2981641|
|     78|   1.0|    24|  0.672634|
|     34|   1.0|    16|-1.9044937|
+-------+------+------+----------+
only showing top 10 rows



In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.9344089097503774


The RMSE described our error in terms of the stars rating column.

### Supply a recommendation to a user:

In [15]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [16]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     12|    11|
|     21|    11|
|     25|    11|
|     38|    11|
|     48|    11|
|     62|    11|
|     64|    11|
|     70|    11|
|     71|    11|
|     72|    11|
|     80|    11|
|     81|    11|
+-------+------+



In [17]:
reccomendations = model.transform(single_user)

In [18]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     70|    11|   2.59325|
|     38|    11| 2.3790333|
|     48|    11| 2.1262581|
|     64|    11| 1.2386378|
|     81|    11| 1.1989366|
|     72|    11|  0.977689|
|     21|    11|0.73006463|
|     12|    11|  0.483051|
|     62|    11|0.39135772|
|     80|    11| 0.2439658|
|     71|    11|-0.3942676|
|     25|    11|-1.7225363|
+-------+------+----------+

